In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-18 11:07 ajna         INFO     Configuração de log efetuada


2025-09-18 11:07:18,071 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL para saida_redex (alias c)
# sql_saida_redex = f"""
# SELECT 
#    id, placa, placaSemirreboque, numeroConteiner, vazioConteiner, codigoRecinto, cpfMotorista, nomeMotorista, dataHoraOcorrencia, direcao
#FROM apirecintos_acessosveiculo 
#WHERE
#    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
#    AND operacao = 'C'
#    AND direcao = 'S'
#    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
#    AND numeroConteiner !=''
#    AND vazioConteiner = 0 
#    """
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# Este '0' indica que o conteiner saiu estufado (cheio) do Redex


# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id, c.placa, c.placaSemirreboque, c.numeroConteiner, c.vazioConteiner,
    c.codigoRecinto, c.cpfMotorista, c.nomeMotorista, c.dataHoraOcorrencia, c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia > c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
    AND c.vazioConteiner = 0
"""


In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,12278020,EWJ9661,FTU5124,WSCU7529693,8931318,2025-09-11 11:14:36,C,E,12014047000193,90643925449,ANDRE TEIXEIRA DA SILVA,,,0
1,12278031,GYS3A17,CPJ4E56,FTAU1429448,8931318,2025-09-11 11:35:32,C,E,32256956000145,16240397822,ROBERTO MARQUES,,152505272092753,0
2,12278030,JNW8G50,DJC6B64,MSDU6992351,8931318,2025-09-11 11:36:54,C,E,32256956000145,30353862851,ROGERIO ANDRADE DA CONCEICAO,,,0
3,12278033,CUD3J75,CUD3881,HMMU6524453,8931318,2025-09-11 11:38:12,C,E,12014047000193,27885045862,REGINALDO MENDONCA DA SILVA,,,0
4,12278032,BYA1H53,BSF3I51,MSMU4709259,8931318,2025-09-11 11:39:32,C,E,32256956000145,38244868813,FABRICIO MATIAS DE ALMEIDA,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22346,12473618,RQN9B79,FFC7J57,HLBU9073569,8931404,2025-09-17 23:52:15,C,E,22029534000183,45317335841,BALBYON CHRYSTIAN RAGOUSSIS FR,35250968067446000410550010014387321080550585,,0
22347,12473619,LPA8318,DDD2300,MRKU6883117,8931404,2025-09-17 23:52:20,C,E,49871560000100,37275802830,ISAAC ALVES DA SILVA,"35250909538989000670550010000644371189449790, ...",,0
22348,12473623,MTU4139,TJY9F85,TGCU5138429,8931404,2025-09-17 23:54:25,C,E,09123539000102,13377642803,EVERALDO ALVES DOS SANTOS,35250972921638000176550030000054501142315996,,0
22349,12473624,MIM3D84,JAD5C73,HLBU9029951,8931404,2025-09-17 23:54:51,C,E,06014292000106,13093591805,BENEDITO APARECIDO VIEIRA,11250906088741002520550010000260381012509484,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,12298718,KLV6414,BWQ3518,PCIU1407726,0,8932796,40681400811,FLAVIO ANTONIO DA SILVA JUNIOR,2025-09-11 11:08:53,S
1,12298850,KMP1F16,BWH1561,CAIU3849360,0,8932796,35147384839,EWERTON HENRIQUE DA SILVA LIMA,2025-09-11 11:12:19,S
2,12298852,MIH1F29,DPC9J07,PCIU1331235,0,8932796,30792512880,LUIZ CARLOS BUFFONI JUNIOR,2025-09-11 11:24:52,S
3,12298855,BTR0863,BXF7628,CAIU6252470,0,8932796,26801536850,RONALDO NASCIMENTO PEDROZA,2025-09-11 11:27:20,S
4,12298857,IUL9J16,ETU4882,PCIU1089630,0,8932796,29229139882,BRUNO RIBEIRO SILVA,2025-09-11 11:29:59,S
...,...,...,...,...,...,...,...,...,...,...
1076,12473481,FEI3I38,FUY9846,TCNU4527572,0,8931404,17898192885,ANTONIO CARLOS BRAGA,2025-09-17 22:42:57,S
1077,12473483,NFV3758,ETU4J55,MRKU4420622,0,8931404,40491493894,PITHER ALBERTO CAVALCANTE,2025-09-17 22:43:56,S
1078,12473560,NKO5G51,FEI3F94,UACU3765876,0,8931404,43147989801,PAULO HENRIQUE DOMINGUES DE AL,2025-09-17 23:23:29,S
1079,12473569,CLH5D40,LXO6909,CAAU6480436,0,8931404,33874139808,OTAVIO JOSE DE SOUZA SILVA,2025-09-17 23:27:25,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/1068 [00:00<?, ?it/s]

Registros: 1068, imagens: 383


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12423418,MTC4J24,FAX1317,HMMU6543221,8931318,2025-09-16 09:34:07,C,E,12014047000193,31704711800,...,12434386,MTC4J24,0,8931404,31704711800,JOSE ADILSON BISPO DOS SANTOS,2025-09-16 08:30:51,S,1.054444,68c97640f614e4f4040efa19
1,12423435,CUD4902,FOE4388,HMMU6790052,8931318,2025-09-16 09:56:49,C,E,12014047000193,10430518803,...,12434419,CUD4902,0,8931404,10430518803,ALDO PASCOAL SOARES JUNIOR,2025-09-16 08:53:56,S,1.048056,68c97640f614e4f4040efa1e
2,12423469,EWJ9661,FTU5124,KOCU4524804,8931318,2025-09-16 11:19:54,C,E,12014047000193,90643925449,...,12434555,EWJ9661,0,8931404,90643925449,ANDRE TEIXEIRA DA SILVA SANTOS,2025-09-16 10:16:34,S,1.055556,68ca98be4b6fcba5bbb13a1d
3,12423491,FEI3I38,FUY9846,HMMU5519489,8931318,2025-09-16 11:54:34,C,E,12014047000193,17898192885,...,12434571,FEI3I38,0,8931404,17898192885,ANTONIO CARLOS BRAGA,2025-09-16 10:22:29,S,1.534722,68ca98be4b6fcba5bbb13a18
4,12423598,KJQ0F31,FUI4356,MOFU6706340,8931318,2025-09-16 17:01:37,C,E,12014047000193,08923508810,...,12435152,KJQ0F31,0,8931404,08923508810,PEDRO DE SOUZA,2025-09-16 16:02:10,S,0.990833,68ca990e8e8f95cb27bf8bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,12471672,GKT4527,DPC9I47,TEMU9039734,8931359,2025-09-17 22:55:42,C,E,58130089000786,36261090890,...,12473783,GKT4527,0,8932709,36261090890,CEZAR DA SILVA BELINI,2025-09-17 22:01:52,S,0.897222,None
1064,12471709,FEI3D72,DTB5A30,MSMU8465660,8931359,2025-09-17 22:56:45,C,E,05457125000169,26009622808,...,12474956,FEI3D72,0,8932778,26009622808,FABIO CHIGOLE,2025-09-17 21:56:00,S,1.012500,68cbf16ab29c25d618a8cfda
1065,12471732,DAO9575,GJI9827,ONEU1068697,8931359,2025-09-17 23:13:27,C,E,58130089000786,22625730854,...,12476188,DAO9575,0,8932798,22625730854,BRUNO SILVA FRANCISCO,2025-09-17 22:03:18,S,1.169167,68cbf16bb29c25d618a8d107
1066,12471774,IHJ2772,CPJ2G59,SEKU4402594,8931359,2025-09-17 23:16:47,C,E,58130089000786,16235567839,...,12476228,IHJ2772,0,8932798,16235567839,VANDERLEI DOS SANTOS SANTANA,2025-09-17 22:10:30,S,1.104722,68cbf16bb29c25d618a8d114


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
498,DBM7C27,FANU3325433,8932796,2025-09-15 18:33:35,8931356,2025-09-16 15:51:24,21.296944,None,
499,DBM7C27,FANU3325433,8932796,2025-09-15 18:49:29,8931356,2025-09-16 15:51:24,21.031944,None,
570,EQY9C81,CGMU5711540,8932761,2025-09-11 12:42:33,8931356,2025-09-17 06:01:20,137.313056,None,
606,CPJ5901,MRKU5998826,8931404,2025-09-17 05:01:47,8931356,2025-09-17 18:13:11,13.190000,None,
637,CUL2G26,MRSU7808759,8931404,2025-09-16 21:08:49,8931356,2025-09-17 22:47:32,25.645278,None,
797,SUE1G56,MNBU0559266,8932761,2025-09-11 18:01:19,8931359,2025-09-14 16:06:08,70.080278,68c7fd2f4b6fcba5bbb103d6,
896,FVK2G86,MNBU4377892,8932761,2025-09-11 15:49:56,8931359,2025-09-16 08:02:01,112.201389,68c976334b6fcba5bbb13537,
915,CMO9C86,MEDU9014456,8932761,2025-09-14 12:29:07,8931359,2025-09-16 10:46:22,46.287500,None,
917,FQJ7611,MEDU9657529,8932761,2025-09-12 16:36:00,8931359,2025-09-16 16:56:34,96.342778,68ca990d8e8f95cb27bf8b3d,
921,FQG2A51,MEDU9091180,8932761,2025-09-14 12:21:37,8931359,2025-09-17 00:06:07,59.741667,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"